# Report - Relationship between the Number of Charging Points and Electromobility in Germany
By: Niklas Markert


## Preparation

In [1]:
# Install Dependencies
%pip install numpy
%pip install pandas
%pip install 'SQLAlchemy==1.4.46'
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import subprocess
from sqlalchemy import create_engine

In [12]:
# Load Data
if not os.path.exists("../data/data.sqlite"):
    subprocess.run(["python", "../data/datapipeline.py"])

con = create_engine('sqlite:///../data/data.sqlite')
df1 = pd.read_sql_table('over_time', con, index_col='Year')
df2 = pd.read_sql_table('by_states', con, index_col='State')

## Introduction

TODO:
* Explanation of the topic
* What is the aim of this report
* What data do we analyze
* ...

## Analysis

TODO:
* Work with code and graphics
* Explain step by step what we are doing
* Analyze the shown data
* ...

## Conclusion
TODO:
* What do we conclude from our findings
* What effects does that have
* ...